In [50]:
import requests
import json

# 추후 혼잡도 데이터 업데이트를 위한 INFO parsing
INFO_URL = 'https://infuser.odcloud.kr/oas/docs?namespace=15071311/v1'
API_KEY = '7x1TU10Ct0763mZVXpBvYSYizKLENWLSWK9K1J6pz0euL7mh7iZhN0TEKp48968l7TFlRh9SXPsO%2BANWRXaDUA%3D%3D'

path_raw = requests.get(INFO_URL)
path_data = json.loads(path_raw.text)

update_path = list(path_data['paths'])[-1] # update 시 이코드 사용

# 지하철 혼잡도 API
CONGESTION_API = f'https://api.odcloud.kr/api{update_path}?page=1&perPage=2000&serviceKey={API_KEY}'

cong_raw = requests.get(CONGESTION_API)
cong_data = json.loads(cong_raw.text)



In [75]:
# 지하철 경로 검색 API
dept = '0201' # 출발지 역코드
dest = '0219' # 도착지 역코드
week = 'DAY' # 주중(DAY), 토요일(SAT), 공휴일(HOL) 공휴일 에러남... 메뉴얼과 코드가 다른거 같음
search_type = 'FASTEST' # (최소 시간)FASTEST, (최소 환승)MINTRF
dept_time = '120000' # (출발 시간)HHmmss

SUBWAY_NAV_API = f'https://apis.data.go.kr/B553766/smt-path/path?serviceKey={API_KEY}&pageNo=1&numOfRows=300&dept_station_code={dept}&dest_station_code={dest}&week={week}&search_type={search_type}&dept_time={dept_time}'

nav_raw = requests.get(SUBWAY_NAV_API)
nav_raw.text
nav_data = json.loads(nav_raw.text)

In [76]:
nav_data

{'data': {'arrv_time': '122300',
  'start_st': '0211',
  'end_st': '0211',
  'week': 'DAY',
  'transfer': 0,
  'route': [{'station_cd': '0211',
    'station_nm': '성수',
    'timestamp': '105400',
    'train_code': '2192',
    'line_num': '2',
    'transfer_loc': None},
   {'station_cd': '0212',
    'station_nm': '건대입구',
    'timestamp': '105600',
    'train_code': '2192',
    'line_num': '2',
    'transfer_loc': None},
   {'station_cd': '0213',
    'station_nm': '구의',
    'timestamp': '105830',
    'train_code': '2192',
    'line_num': '2',
    'transfer_loc': None},
   {'station_cd': '0214',
    'station_nm': '강변',
    'timestamp': '110000',
    'train_code': '2192',
    'line_num': '2',
    'transfer_loc': None},
   {'station_cd': '0215',
    'station_nm': '잠실나루',
    'timestamp': '110230',
    'train_code': '2192',
    'line_num': '2',
    'transfer_loc': None},
   {'station_cd': '0216',
    'station_nm': '잠실',
    'timestamp': '110430',
    'train_code': '2192',
    'line_num': '2',

In [20]:
'SERVICE ERROR' in nav_raw.text

True

In [26]:
list(path_data['paths'])[-3:]

['/15071311/v1/uddi:a5158b81-27c7-4151-ba6c-b912a6f13d39',
 '/15071311/v1/uddi:70e3a3d3-0872-4828-8234-f0bca459b44f',
 '/15071311/v1/uddi:b3803d43-ffe3-4d17-9024-fd6cfa37c284']

In [27]:
# API 끼리 역코드가 맞는지 확인
search = [int(dept), int(dest)]

for i in range(cong_data['currentCount']):
  if cong_data['data'][i]['역번호'] in search:
    print(cong_data['data'][i]['역명'], cong_data['data'][i]['역번호'])

강변 214
강변 214
어린이대공원 2728
어린이대공원 2728
강변 214
강변 214
어린이대공원 2728
어린이대공원 2728
강변 214
강변 214
어린이대공원 2728
어린이대공원 2728


In [74]:
# 상선 / 하선, 내선 / 외선 넘버링 확인
for i in cong_data['data']:
  if i['역명'] == '삼성':
    print(i['역명'], i['역번호'], i['구분'], i['10시00분'])

삼성 219 내선 34.2
삼성 219 외선 16.1
삼성 219 내선 26.2
삼성 219 외선 19.9
삼성 219 내선 17.5
삼성 219 외선 13.2


In [53]:
data = nav_data['data']['route']

clss_list = []

for i in range(len(data) - 1):
  if int(data[i]['line_num']) == int(data[i + 1]['line_num']):
    if int(data[i]['line_num']) in [2, 13, 14]: # 2호선
      if int(data[i]['station_cd']) < int(data[i + 1]['station_cd']):
        clss_list.append(3) # 내선
      else:
        clss_list.append(4) # 외선
    elif int(data[i]['line_num']) in [1, 4]: # 1호선, 4호선
      if int(data[i]['station_cd']) < int(data[i + 1]['station_cd']):
        clss_list.append(1) # 상선
      else:
        clss_list.append(2) # 하선
    else: # 그 외 호선
      if int(data[i]['station_cd']) < int(data[i + 1]['station_cd']):
        clss_list.append(2) # 하선
      else:
        clss_list.append(1) # 상선
  else:
    continue # 환승하기전은 계산하지 않음

clss_list

[2, 3, 3]

In [60]:
[row for row in nav_data['data']['route'] if row['transfer_loc'] == None]

[{'station_cd': '2728',
  'station_nm': '어린이대공원',
  'timestamp': '120340',
  'train_code': '7187',
  'line_num': '7',
  'transfer_loc': None},
 {'station_cd': '0212',
  'station_nm': '건대입구',
  'timestamp': '121400',
  'train_code': '2218',
  'line_num': '2',
  'transfer_loc': None},
 {'station_cd': '0213',
  'station_nm': '구의',
  'timestamp': '121630',
  'train_code': '2218',
  'line_num': '2',
  'transfer_loc': None},
 {'station_cd': '0214',
  'station_nm': '강변',
  'timestamp': '121800',
  'train_code': '2218',
  'line_num': '2',
  'transfer_loc': None}]

In [54]:
for row in nav_data['data']['route'][:-1]:
  if row['transfer_loc'] != None:
    continue
  print(row)

{'station_cd': '2728', 'station_nm': '어린이대공원', 'timestamp': '120340', 'train_code': '7187', 'line_num': '7', 'transfer_loc': None}
{'station_cd': '0212', 'station_nm': '건대입구', 'timestamp': '121400', 'train_code': '2218', 'line_num': '2', 'transfer_loc': None}
{'station_cd': '0213', 'station_nm': '구의', 'timestamp': '121630', 'train_code': '2218', 'line_num': '2', 'transfer_loc': None}


In [7]:
nav_data['data']['route']

[{'station_cd': '2728',
  'station_nm': '어린이대공원',
  'timestamp': '120340',
  'train_code': '7129',
  'line_num': '7',
  'transfer_loc': None},
 {'station_cd': '2729',
  'station_nm': '건대입구',
  'timestamp': '120540',
  'train_code': '7129',
  'line_num': '7',
  'transfer_loc': '8-4'},
 {'station_cd': '0212',
  'station_nm': '건대입구',
  'timestamp': '121200',
  'train_code': '2180',
  'line_num': '2',
  'transfer_loc': None},
 {'station_cd': '0213',
  'station_nm': '구의',
  'timestamp': '121430',
  'train_code': '2180',
  'line_num': '2',
  'transfer_loc': None},
 {'station_cd': '0214',
  'station_nm': '강변',
  'timestamp': '121600',
  'train_code': '2180',
  'line_num': '2',
  'transfer_loc': None}]

In [38]:
a = [code['station_cd'] for code in nav_data['data']['route']]
a

['2728', '2729', '0212', '0213', '0214']

In [37]:
a = [code['station_cd'] for code in nav_data['data']['route'] if code['transfer_loc'] == None]
a

['2728', '0212', '0213', '0214']

In [123]:
update_path = list(path_data['paths'])[2] # update 시 이코드 사용

# 지하철 혼잡도 API
CONGESTION_API = f'https://api.odcloud.kr/api{update_path}?page=1&perPage=2000&serviceKey={API_KEY}'

cong_raw = requests.get(CONGESTION_API)
cong_data = json.loads(cong_raw.text)

In [135]:
a, b = list(cong_data['data'][0].items())[:-6][0]

In [133]:
import numpy as np

a = [[1, 2, 3], [4, 5, 6]]

x = np.array([y for y in a])
x

array([[1, 2, 3],
       [4, 5, 6]])

In [147]:
from datetime import date, datetime

datetime(2012, 3, 4, 1, 2, 3) < datetime.now()

True

In [154]:
for data in cong_data['data']:
  for t, c in list(data.items())[:-6]:
    print(t, c)

10시00분 26.6
10시30분 23.4
11시00분 22.4
11시30분 22.1
12시00분 21.5
12시30분 27.2
13시00분 21.2
13시30분 24.1
14시00분 21.7
14시30분 18.7
15시00분 18.5
15시30분 18.4
16시00분 16.4
16시30분 17.7
17시00분 16.7
17시30분 24.1
18시00분 30.8
18시30분 28.7
19시00분 20.0
19시30분 15.1
20시00분 11.1
20시30분 12.0
21시00분 12.5
21시30분 10.1
22시00분 10.4
22시30분 16.4
23시00분 7.8
23시30분 6.1
5시30분 6.2
6시00분 14.6
6시30분 13.4
7시00분 26.8
7시30분 43.3
8시00분 53.6
8시30분 70.9
9시00분 40.4
9시30분 38.0
10시00분 13.9
10시30분 15.2
11시00분 14.6
11시30분 14.2
12시00분 17.6
12시30분 19.6
13시00분 20.3
13시30분 20.5
14시00분 25.4
14시30분 22.0
15시00분 30.1
15시30분 24.4
16시00분 33.8
16시30분 36.7
17시00분 43.8
17시30분 49.7
18시00분 84.0
18시30분 65.1
19시00분 37.5
19시30분 26.5
20시00분 25.5
20시30분 26.4
21시00분 26.3
21시30분 21.1
22시00분 32.1
22시30분 20.3
23시00분 12.6
23시30분 10.3
5시30분 9.1
6시00분 8.6
6시30분 9.3
7시00분 15.2
7시30분 29.3
8시00분 27.7
8시30분 19.0
9시00분 18.5
9시30분 15.1
10시00분 20.2
10시30분 21.9
11시00분 20.8
11시30분 20.0
12시00분 21.7
12시30분 24.4
13시00분 21.7
13시30분 23.7
14시00분 21.5
14시30분 20.7
15시00분 20.3
15시3

In [3]:
from app.src.make_model import st_dict

st_list = list(st_dict())

/Users/kkangjun/Desktop/Study/CodeStates/Project/Project4/codestates_Project4/app/src/make_model.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataset = pd.read_sql("SELECT * FROM train_cong", conn)


In [11]:
a = '2023-04-09T14:30'
format = '%Y-%m-%dT%H:%M'

from datetime import datetime, date

b = datetime.strptime(a, format)

In [14]:
a.split('T')[1].replace(':','')

'1430'

In [13]:
b.weekday()

6

In [20]:
s = '500'

w = s.zfill(4) + '00'

In [22]:
int(w[:4])

500

In [58]:
a = {'1':[1, 2], '2':[3, 4]}

for i in a.values():
  print(type(i))

<class 'list'>
<class 'list'>


In [2]:
from tensorflow.keras.models import load_model
model = load_model('cong_app/templates/latest_model.h5')

In [22]:
from cong_app.src.make_model import dataset_load

dataset = dataset_load()

/Users/kkangjun/Desktop/Study/CodeStates/Project/Project4/codestates_Project4/cong_app/src/make_model.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataset = pd.read_sql("SELECT * FROM train_cong", conn)


In [67]:
dd = dataset[['st_name', 'st_code']].drop_duplicates()
dd

,st_name,st_code
0,서울역,150
74,시청,151
148,종각,152
222,종로3가,153
296,종로5가,154
...,...,...
20646,남한산성입구,2823
20720,단대오거리,2824
20794,신흥,2825
20868,수진,2826


In [68]:
d = {}
for a, b in zip(dd['st_name'], dd['st_code']):
  try:
    if d[a]:
      temp = d[a]
      del d[a]
      # 원래 있던 키값 변경
      if temp < 200:
        d[a+'(1호선)'] = temp
      elif temp < 300:
        d[a+'(2호선)'] = temp
      elif temp < 400:
        d[a+'(3호선)'] = temp
      elif temp < 2500:
        d[a+'(4호선)'] = temp
      elif temp < 2600:
        d[a+'(5호선)'] = temp
      elif temp < 2700:
        d[a+'(6호선)'] = temp
      elif temp < 2700:
        d[a+'(7호선)'] = temp
      elif temp < 2700:
        d[a+'(8호선)'] = temp
      
      # 새로운 키 추가
      if b < 200:
        d[a+'(1호선)'] = b
      elif b < 300:
        d[a+'(2호선)'] = b
      elif b < 400:
        d[a+'(3호선)'] = b
      elif b < 2500:
        d[a+'(4호선)'] = b
      elif b < 2600:
        d[a+'(5호선)'] = b
      elif b < 2700:
        d[a+'(6호선)'] = b
      elif b < 2700:
        d[a+'(7호선)'] = b
      elif b < 2700:
        d[a+'(8호선)'] = b
  except:
    d[a] = b

In [71]:
d['사당(4호선)']

433